In [7]:
import numpy as np 
import pandas as pd

In [8]:
# criação do df com os resultados do melhor guloso para todos os grafos simples analisados
gul_files = [
    "resultsGulSimplesADJ_n100_sumario.csv",
    "resultsGulSimplesADJ_n500_sumario.csv",
    "resultsGulSimplesADJ_n1000_sumario.csv",
    "resultsGulSimplesADJ_n1500_sumario.csv"
]

df_gul_all = pd.concat([pd.read_csv(f) for f in gul_files], ignore_index=True)
print(df_gul_all.columns.tolist())

['n_config', 'tempo_melhor', 'v_config', 'N_real', 'melhor_chi', 'instancia', 'heuristica_vencedora', 'M_real', 'p_config']


In [14]:
# NOTA: pode ser útil organizar as colunas dos CSV's melhor

# criação do df com os resultados do genético para todos os grafos simples analisados
gen_files = [
    "results_GA_Final_corrigido_n100.csv",
    "results_GA_Final_corrigido_n500.csv",
    "results_GA_Final_n1000.csv",
    "results_GA_Final_corrigido_n1500.csv"
]

df_gen_all = pd.concat([pd.read_csv(f) for f in gen_files], ignore_index=True)
print(df_gen_all.columns.tolist())

['N', 'p', 'n', 'M', 'v', 'mean_time', 'se_time', 'instancia', 'mean_chi', 'se_chi']


In [15]:
# merge dos dataframes para observar resultados em número de cores e tempo do método
df_comparacao = pd.merge(df_gul_all, df_gen_all, on="instancia")

# mean_chi é do genético e melhor_chi é do guloso
df_comparacao['diff_chi'] = df_comparacao['mean_chi'] - df_comparacao['melhor_chi']

# mean_time é do genético e tempo_melhor é do guloso
df_comparacao['diff_tempo'] = df_comparacao['mean_time'] - df_comparacao['tempo_melhor']

# porcentagem de melhora em termos de cores para o genético
df_comparacao['pct_melhora_chi'] = (df_comparacao['diff_chi'] / df_comparacao['melhor_chi']) * 100

# "desaceleração" com o uso do genético
df_comparacao['slowdown_tempo'] = df_comparacao['mean_time'] / df_comparacao['tempo_melhor']

In [16]:
estatisticas_chi = df_comparacao[['mean_chi', 'melhor_chi', 'diff_chi', 'pct_melhora_chi']].describe()
estatisticas_tempo = df_comparacao[['mean_time', 'tempo_melhor', 'diff_tempo']].describe()

print("--- Estatísticas de Cores (Chi) ---")
print(estatisticas_chi)
print("\n--- Estatísticas de Tempo ---")
print(estatisticas_tempo)

vitorias_gen = (df_comparacao['mean_chi'] < df_comparacao['melhor_chi']).sum()
empates = (df_comparacao['mean_chi'] == df_comparacao['melhor_chi']).sum()
vitorias_gul = (df_comparacao['mean_chi'] > df_comparacao['melhor_chi']).sum()

print(f"\nResumo de Performance (Cores):")
print(f"Genético melhor: {vitorias_gen}")
print(f"Empates: {empates}")
print(f"Guloso melhor: {vitorias_gul}")

--- Estatísticas de Cores (Chi) ---
          mean_chi   melhor_chi    diff_chi  pct_melhora_chi
count   238.000000   238.000000  238.000000       238.000000
mean    650.805882   650.189076    0.616807        -1.862691
std     524.184427   523.350003   10.003293         8.664965
min      17.200000    25.000000  -65.600000       -43.733333
25%     100.000000   112.250000    0.000000         0.000000
50%     500.000000   500.000000    0.000000         0.000000
75%    1000.000000  1000.000000    0.000000         0.000000
max    1500.000000  1500.000000   35.200000        19.875776

--- Estatísticas de Tempo ---
         mean_time  tempo_melhor   diff_tempo
count   238.000000    238.000000   238.000000
mean    390.747383      1.645852   389.101531
std     632.799856      4.203971   629.293255
min       0.090160      0.000041     0.090119
25%       2.090682      0.003251     2.080311
50%      60.742720      0.057619    60.463674
75%     569.604975      1.084795   567.268292
max    2601.7579